# The Canada/US Border Crossing Dataset


*Exploration of whether the variance in border crossings since 1972 is related to socio-economic indices*

## Load Data

**Import Libraries**

In [1]:
import pandas as pd #working with dataframes

**Load Data**

In [2]:
# Data accessed from: https://open.canada.ca/data/en/dataset/6e438d59-880a-4f5e-827d-6c35243cf10a
# Data accessed on: 2019-05-24

border_df_000 = pd.read_csv('24100002.csv')

C:\DataScience\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Vet Data

In [7]:
border_df_000.head()

,REF_DATE,GEO,DGUID,Trip characteristics,Length of stay,Mode of transportation,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1972-01,Canada,2016A000011124,Total vehicles entering Canada,"Length of stay, total","Mode of transportation, total",Number,223,units,0,v129299,1.1.1.1,1427026.0,NaN,NaN,NaN,0
1,1972-01,Canada,2016A000011124,Total United States vehicles entering,"Length of stay, total","Mode of transportation, total",Number,223,units,0,v129300,1.2.1.1,616944.0,NaN,NaN,NaN,0
2,1972-01,Canada,2016A000011124,Total United States vehicles entering,"Length of stay, total",Automobiles,Number,223,units,0,v129301,1.2.1.2,557108.0,NaN,NaN,NaN,0
3,1972-01,Canada,2016A000011124,Total United States vehicles entering,"Length of stay, total",Trucks,Number,223,units,0,v129305,1.2.1.3,59491.0,NaN,NaN,NaN,0
4,1972-01,Canada,2016A000011124,Total United States vehicles entering,"Length of stay, total",Other vehicles,Number,223,units,0,v129306,1.2.1.4,345.0,NaN,NaN,NaN,0


In [8]:
# find the number of unique responses in each column

border_df_000.nunique()

REF_DATE                    567
GEO                         142
DGUID                        10
Trip characteristics          3
Length of stay                4
Mode of transportation        4
UOM                           1
UOM_ID                        1
SCALAR_FACTOR                 1
SCALAR_ID                     1
VECTOR                     2130
COORDINATE                 4260
VALUE                     78833
STATUS                        1
SYMBOL                        0
TERMINATED                    0
DECIMALS                      1
dtype: int64

*Any feature that has 1 or less unique elements can be removed, since there is no variance to explore*

**Drop Columns**

In [ ]:
# Drop columns that have 1 or less unique elements


In [24]:
border_df_000.nunique()

REF_DATE                    567
GEO                         142
DGUID                        10
Trip characteristics          3
Length of stay                4
Mode of transportation        4
UOM                           1
UOM_ID                        1
SCALAR_FACTOR                 1
SCALAR_ID                     1
VECTOR                     2130
COORDINATE                 4260
VALUE                     78833
STATUS                        1
SYMBOL                        0
TERMINATED                    0
DECIMALS                      1
dtype: int64

In [18]:
column_unique[0].index

AttributeError: 'numpy.int64' object has no attribute 'index'

In [25]:
drop_nonunique = [] # create an empty list to put columns to be dropped in
column_unique = border_df_000.nunique()

for i in range(len(border_df_000.nunique())):
    if column_unique[i] <= 1:
        drop_nonunique.append(column_unique.index[i])

drop_nonunique    

['UOM',
 'UOM_ID',
 'SCALAR_FACTOR',
 'SCALAR_ID',
 'STATUS',
 'SYMBOL',
 'TERMINATED',
 'DECIMALS']